In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


## Gas accretion rate, peb iso and M0 new vs old prescription

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,6))
t = np.linspace(0.3, 10)
params = code_gas.Params(M_dot_star=0, H_r_model="Liu_mixed", star_mass=1*const.M_sun.to(u.M_earth).value)
params2 = code_gas.Params(M_dot_star=0, H_r_model="Liu_mixed", star_mass=0.1*const.M_sun.to(u.M_earth).value)
params_old = code_gas.Params(M_dot_star=None,  H_r_model="Lambrechts_mixed", star_mass=0.1*const.M_sun.to(u.M_earth).value)
params_old1 = code_gas.Params(M_dot_star=None,  H_r_model="Lambrechts_mixed", star_mass=1*const.M_sun.to(u.M_earth).value)


# gas accretion rate
axs[0].loglog(t, (M_dot_star_t(t)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color = "black", label = "Hartmann et al. 2016")
axs[0].loglog(t, (M_dot_star_t_Mstar(t, params)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color = "green", label = "Liu et al. 2019, $M_{*}$="+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[0].loglog(t, (M_dot_star_t_Mstar(t, params2)*u.M_earth/u.Myr).to(u.M_sun/u.yr), color = "red", label = "Liu et al. 2019, $M_{*}$="+str(params2.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[0].set_xlabel("t [Myr]", size = 15)
axs[0].set_ylabel("$\dot{M}_* [M_{\odot}/yr]$", size = 15)
axs[0].tick_params(axis='both', which = 'major', size = 10, direction='in')
axs[0].tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs[0].legend()
axs[0].set_title("Mass accretion rate onto the star", size = 15)
print("L_star", params.star_luminosity)
print("L_star", params2.star_luminosity)
print("L_sun", (const.L_sun.cgs).value *erg_s_to_au_M_E_Myr)
#pebble isolation mass
r = np.geomspace(1e-1, 40, 1000)
t0 = 0.1

# mass 0.1 M_sun
axs[1].loglog(r, M_peb_iso(r, t0, params_old), color = "black", label = "Danti et al. 2025, $M_{*}$="+str(params_old.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[1].loglog(r, M_peb_iso_new(r, t0, params2), color = "gold", label = "Liu et al. 2019, $M_{*}$="+str(params2.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
#axs[1].loglog(r, M_peb_iso_Bitsch(r, t0, params_old), color = "blue", label = "Bitsch et al. 2018, $M_{*}$="+str(params_old.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[1].loglog(r, M_peb_iso_Bitsch(r, t0, params2), color = "violet", label = "Bitsch et al. 2018, $M_{*}$="+str(params2.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")

#mass 1 M_sun
axs[1].loglog(r, M_peb_iso_new(r, t0, params), color = "green", label = "Liu et al. 2019, $M_{*}$="+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[1].loglog(r, M_peb_iso(r, t0, params_old1), color = "grey", label = "Danti et al. 2025, $M_{*}$="+str(params_old1.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")


axs[1].set_xlabel("r [AU]", size = 15)
axs[1].set_ylabel("$M [M_{\oplus}]$", size = 15)
axs[1].tick_params(axis='both', which = 'major', size = 10, direction='in')
axs[1].tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs[1].legend()
axs[1].set_title("Pebble isolation mass", size = 15)

#initial mass

axs[2].loglog(r, M0_pla(r, t0, sigma_gas_steady_state(r, t0, params_old), params_old), color = "black", label = "Danti et al. 2025")
axs[2].loglog(r, M0_pla_Mstar(r, t0, sigma_gas_steady_state(r, t0, params), params), color = "green", label = "Liu et al. 2019, $M_{*}$="+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[2].loglog(r, M0_pla_Mstar(r, t0, sigma_gas_steady_state(r, t0, params2),params2), color = "gold", label = "Liu et al. 2019, $M_{*}$="+str(params2.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$")
axs[2].set_xlabel("r [AU]", size = 15)
axs[2].set_ylabel("$M [M_{\oplus}]$", size = 15)
axs[2].tick_params(axis='both', which = 'major', size = 10, direction='in')
axs[2].tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs[2].legend()
axs[2].set_title("Initial mass", size = 15)

plt.savefig("figures/M_dot_star_peb_iso_M0.png", bbox_inches='tight')

In [ ]:
par = code_gas.Params(M_dot_star=0, H_r_model="Liu_mixed", star_mass=10*const.M_sun.to(u.M_earth).value)
para = code_gas.Params(M_dot_star=0, H_r_model="Liu_mixed", star_mass=0.1*const.M_sun.to(u.M_earth).value)
print("L_star 10", par.star_luminosity*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun), "M_star", par.star_mass)
print("L_star 0.1", para.star_luminosity*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun), "M_star", para.star_mass)
print("L_sun", (const.L_sun.cgs).value *erg_s_to_au_M_E_Myr*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun))
def L_star(M_star):
    return ((M_star/M_sun_M_E)**(3/2)+0.001)*(const.L_sun.cgs).value*erg_s_to_au_M_E_Myr
print("L_star 0.1 formula", L_star((0.1*const.M_sun).to(u.M_earth).value)*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun))

In [ ]:
1-((2e-3-(0.01)**(3/2))/(1-(0.01)**(3/2)))

In [ ]:
def L_star(M_star):
    return ((M_star/M_sun_M_E)**(3/2)+0.001)*(const.L_sun.cgs).value*erg_s_to_au_M_E_Myr

def L_linear(M_star):
    return M_star/M_sun_M_E*(const.L_sun.cgs).value*erg_s_to_au_M_E_Myr 
def L_quadratic(M_star):
    return (M_star/M_sun_M_E)**2*(const.L_sun.cgs).value*erg_s_to_au_M_E_Myr

M_sun_M_E = (const.M_sun.to(u.M_earth)).value
erg_s_to_au_M_E_Myr = (1*u.erg/u.s).to(u.au**2*u.M_earth/u.Myr**3).value

fig, axs = plt.subplots(1,1, figsize = (6,6))
m = np.linspace(0.01, 1, 100000)*M_sun_M_E
axs.loglog(m*u.M_earth.to(u.M_sun), L_star(m)*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun), color = "black")
axs.loglog(m*u.M_earth.to(u.M_sun), L_linear(m)*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun), color = "green", linestyle = "--")
axs.loglog(m*u.M_earth.to(u.M_sun), L_quadratic(m)*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun), color = "green", linestyle = "--")
axs.set_xlabel("$M_{*} [M_{\odot}]$", size = 15)
axs.set_ylabel("$L_{*} [L_{\odot}]$", size = 15)    
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.set_title("Stellar luminosity", size = 15)
#axs.set_ylim(2e-6,2)
print("L_star(0.1) = ", L_star((0.1*const.M_sun).to(u.M_earth).value)*(u.au**2*u.M_earth/u.Myr**3).to(u.L_sun))
plt.savefig("figures/L_star.png", bbox_inches='tight')


In [ ]:
fig, axs = plt.subplots(1,1, figsize = (6,6))
t = np.linspace(0.1, 5)
params = code_gas.Params(H_r_model="Liu_mixed",star_mass= 0.1*const.M_sun.to(u.M_earth).value)
params2 = code_gas.Params(H_r_model="Liu_mixed",M_dot_star=0, star_mass=0.1*const.M_sun.to(u.M_earth).value)
t0 = 0.2
a_p0 = np.geomspace(1e-2, 1e2)
m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params), params)
m0_old = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params2), params2)
m0_new = M0_pla_Mstar(a_p0, t0, sigma_gas_steady_state(a_p0, t, params2), params2)
m0_new_new = M0_pla_Mstar(a_p0, t0, sigma_gas_steady_state(a_p0, t, params), params)


axs.loglog(a_p0, m0_old, color = "gold", label = str(params2.H_r_model)+", M0 old, Mdot = Hartmann 2016")
axs.loglog(a_p0, m0, color = "green", label = str(params2.H_r_model)+", M0 old, Mdot = Liu et. al 2019b")
axs.loglog(a_p0, m0_new, color = "violet", label = str(params.H_r_model)+", M0 Liu et. al 2019b, Mdot = Hartmann 2016")
axs.loglog(a_p0, m0_new_new, color = "navy", label = str(params.H_r_model)+", M0, Mdot Liu et. al 2019b")

axs.loglog(a_p0, M_peb_iso(a_p0,t, params), color = 'black', label = "m iso")
# params_flux = code_gas.Params( H_r_model='irradiated')
# params_alpha_flux = code_gas.Params( H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5)
# params_alpha_flux_alphafrag = code_gas.Params( H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1)

# m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux), params_alpha_flux)
# m03 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux_alphafrag), params_alpha_flux_alphafrag)

# axs.loglog(a_p0, m02, color = "blue", label = str(params_alpha_flux.H_r_model))
# axs.loglog(a_p0, m03, color = "red", label = str(params_alpha_flux_alphafrag.H_r_model))
axs.set_xlabel("r [AU]", size = 15)
axs.set_ylabel("M [$M_{\oplus}$]", size = 15)
axs.tick_params(axis='both', which = 'major', size = 10, direction='in')
axs.tick_params(axis='both', which = 'minor', size = 8, direction='in')
axs.legend()
axs.set_title("Intial mass")

In [ ]:
print((0.0245)**(14/3)*(const.M_sun.cgs*const.G.cgs*2.34*const.m_p.cgs/(const.k_B.cgs*170*u.K*(1*u.au).to(u.cm)))**(7/3))
params = code_gas.Params()
print(iceline_irr(10, 170, params))

In [ ]:
print((0.019)**(20/9)*(const.M_sun.cgs*const.G.cgs*2.34*const.m_p.cgs/(const.k_B.cgs*170*u.K*(1*u.au).to(u.cm)))**(10/9))
params = code_gas.Params(M_dot_star=(1e-8*u.M_sun/u.yr).to(u.M_earth/u.Myr).value)
#params = code_gas.Params()
print(iceline_visc(5, 170, params))

In [ ]:
Gauss_to_au_M_E_myr = (1*u.cm**(-1/2)*u.g**(1/2)/u.s).to(u.au**(-1/2)*u.M_earth**(1/2)/u.Myr).value
def r_m (b,r,m,mdot):
    return 3.2e-4*(b/Gauss_to_au_M_E_myr)**(4/7)*(r/(const.R_sun).to(u.au).value)**(12/7)*(m/(const.M_sun).to(u.M_earth).value)**(-1/7)*(mdot/((1e-8*u.M_sun/u.yr).to(u.M_earth/u.Myr)).value)**(-2/7)

params = code_gas.Params(M_dot_star=(1e-8*u.M_sun/u.yr).to(u.M_earth/u.Myr).value)
print(r_magnetic_cavity(0.1, params))
print(r_m(params.star_magnetic_field, params.star_radius, params.star_mass, params.M_dot_star))
print(params.star_magnetic_field/Gauss_to_au_M_E_myr)

In [ ]:
print("iceline surfheat tin", 0.498*(2e-7/1e-8)**(4/9))
print("iceline surfheat tfin", 0.498*(3.2e-9/1e-8)**(4/9))

print("iceline midheat tin", (1/1e-2)**(2/9)*(1/0.5)**(2/9)*0.498*(2e-7/1e-8)**(4/9))
print("iceline midheat tfin", (1/1e-2)**(2/9)*(1/0.5)**(2/9)*0.498*(3.2e-9/1e-8)**(4/9))


In [ ]:
params = code_gas.Params(H_r_model="irradiated")
params_visc = code_gas.Params(H_r_model="Lambrechts_mixed")
params_visc_mid = code_gas.Params(H_r_model="Lambrechts_mixed", epsilon_el=1, epsilon_heat=1)
print("iceline irr", iceline(0.1, 170, params))
print("iceline surfheat t0", iceline_visc(0.1, 170, params_visc))
print("iceline surfheat tfin", iceline_visc(5, 170, params_visc))
print("iceline midheat t0", iceline_visc(0.1, 170, params_visc_mid))
print("iceline midheat tfin", iceline_visc(5, 170, params_visc_mid))

In [ ]:

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                'epsilon_el': 1,
                'epsilon_heat': 1,
                }

t_initial = 0.1
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.1
t_fin = 5
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }

params = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed')
m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params), params)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)
print(iceline(sim_params.t_in, 170, params))


In [ ]:

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }

params_ida_ext_2 = code_gas.Params(**params_dict, H_r_model='irradiated')
# params_ida_ext_3 = code_gas.Params(**params_dict, H_r_model='Ida_mixed', epsilon_el=1, epsilon_heat=1, alpha =1e-3)

# params_lam_ext_2 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-2)
# params_lam_ext_3 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-3)


# m0_lam_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_2), params_lam_ext_2)
# sim_params_lam_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_2)
# m0_lam_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_3), params_lam_ext_3)
# sim_params_lam_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_3)

m0_ida_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_2), params_ida_ext_2)
sim_params_ida_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_2)
# m0_ida_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_3), params_ida_ext_3)
# sim_params_ida_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_3)

peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

# sim_lam_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_2, sim_params=sim_params_lam_ext_2)
# sim_lam_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_3, sim_params=sim_params_lam_ext_3)
# sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)
# sim_ida_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_3, sim_params=sim_params_ida_ext_3)

sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)

# ## to run the sim and profile execution time and nr of calls
# def run_simulation():
#     sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)
#     #sim_ida_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_3, sim_params=sim_params_ida_ext_3)

# # Profile the function
# profiler = cProfile.Profile()
# profiler.enable()
# run_simulation()
# profiler.disable()

# # Create a Stats object and sort the output by total time
# stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)

# # Print the top 10 functions that take the longest time
# stats.print_stats(10)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
axs.loglog(sim_ida_ext_2.time, sim_ida_ext_2.sigma_gas[0])
print(sim_ida_ext_2.sigma_gas[0])

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
pos = np.geomspace(1e-2, 100, num = 10)
t = np.geomspace(1e-1, 10, num = 10)
t0 = 0.1
params = code_gas.Params(**params_dict)
axs.loglog(pos, H_R_irr(pos, params_ida_ext_2))

In [ ]:
erg_s_to_au_M_E_Myr = (1*u.erg/u.s).to(u.au**2*u.M_earth/u.Myr**3).value
L = (const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3)
M = (const.M_sun).to(u.M_earth)
r = 1*u.au
print((const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3))
print((const.M_sun).to(u.M_earth))
params = code_gas.Params(**params_dict, star_luminosity= L, star_mass = M)
print( "in au",0.024*(L)**(1/7)*(M)**(-4/7)*(r)**(2/7))
print(H_R_irr(1, params))

In [ ]:
fig,axs = plt.subplots(2,2, figsize = (18,18))
label = ["no filtering", "filtering"]


# plot.plot_growth_track_timescale(fig, axs[0], sim_filter_irr, params_irr, sim_params_irr, migration = True,
#                                     cmap = mpl.colormaps["inferno"].reversed(), add_ylabel=True , 
#                                         add_cbar= True , nofilter=True if i ==0 else False
#                                         )

# plot.plot_growth_track_timescale(fig, axs[0,0], sim_lam_ext_3, params_lam_ext_3, sim_params_lam_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[0,1], sim_lam_ext_2, params_lam_ext_2, sim_params_lam_ext_2, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=True, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[1,0], sim_ida_ext_3, params_ida_ext_3, sim_params_ida_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

plot.plot_growth_track_timescale(fig, axs[1,1], sim_ida_ext_2, params_ida_ext_2, sim_params_ida_ext_2, migration = True,
                                cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
                                    add_cbar=True, nofilter=False
                                    )
for p in range(sim_params_ida_ext_2.nr_planets-1,-1,-1):
    isolation_mass = M_peb_iso(sim_ida_ext_2.position[p].value, sim_ida_ext_2.time.value, params_ida_ext_2)
    iso_idx = np.argmax(sim_ida_ext_2.mass[p].value > isolation_mass)
    axs[1,0].axvline(sim_ida_ext_2.time[iso_idx].value, color = 'lightblue', linestyle = '--')

axs[1,0].scatter(sim_ida_ext_2.time.to(u.Myr), sim_ida_ext_2.dR_dt[0], color = 'black')
axs[1,0].set_xscale('log')
axs[1,0].set_xlabel('t [Myr]', size = 20)
axs[1,0].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)

axs[0,1].scatter(sim_ida_ext_2.position.to(u.au)[0], sim_ida_ext_2.dR_dt[0], color = 'black')
axs[0,1].set_xscale('log')
axs[0,1].set_xlabel('R [AU]', size = 20)
axs[0,1].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)
print("dr/dt", sim_ida_ext_2.dR_dt[0])
print("r/rdot", sim_ida_ext_2.position.to(u.au)[0]/sim_ida_ext_2.dR_dt[0])
print("step size [yr]", (sim_params_ida_ext_2.step_size*u.Myr))
print("time vector[Myr]", (sim_ida_ext_2.time*u.Myr))
print(sim_ida_ext_2.time.size)
print(np.where(sim_params_ida_ext_2.step_size*u.Myr>(sim_ida_ext_2.position[0]/np.abs(sim_ida_ext_2.dR_dt[0])))[0])
print("min incremento", np.min(sim_ida_ext_2.position[0]/np.abs(sim_ida_ext_2.dR_dt[0])))
print("min incremento massa", np.min(sim_ida_ext_2.mass[0]/np.abs(sim_ida_ext_2.dM_dt[0])))

axs[0,0].scatter(sim_ida_ext_2.time.to(u.Myr), sim_ida_ext_2.dM_dt[0], color = 'black')
axs[0,0].set_xscale('log')
axs[0,0].set_xlabel('t [Myr]]', size = 20)
axs[0,0].set_ylabel('$\dot{M}$ [$M_{\oplus}$/Myr]', size = 20)
axs[0,0].axhline(sim_params_ida_ext_2.step_size, color = 'red', linestyle = '--')
axs[0,0].set_xlim(0.2, 0.3)
axs[1,0].set_xlim(0.2, 0.3)
for i in range(2):
    for j in range(2):
        axs[i,j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
        axs[i,j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
        #axs[i,j].set_xlim(4e-3, 1e2)
        # plot.kepler11(axs[i,j], color = 'black')
        # plot.solar_system(axs[i,j], color = 'green')



plt.savefig("figures/t_fixed_adaptive_2au", bbox_inches='tight')

## DIFFERENT MODELS PLOT

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'iceline_radius': None,
                'v_frag':(1*u.m/u.s).to(u.au/u.Myr).value,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                }

t_in = 0.2
t_fin = 5
# a_p0_in = np.geomspace(3, 0.1, num=5)
# # Combine the single value 20 with the generated array
# a_p0 = np.concatenate(([20], a_p0_in))
a_p0 = np.geomspace(60, 0.1, num = 5)
print(a_p0)
t0= ([t_in] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work

sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

# params_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=False, iceline_alpha_frag_change=False)
# params_alpha_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=False)
# params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=True)

params_flux = code_gas.Params(**params_dict, H_r_model='irradiated')
params_alpha_flux = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5)
params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1)

m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_flux), params_flux)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)

m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux), params_alpha_flux)
sim_params2 = code_gas.SimulationParams(**sim_params_dict, m0 = m02)

m03 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux_alphafrag), params_alpha_flux_alphafrag)
sim_params3 = code_gas.SimulationParams(**sim_params_dict, m0 = m03)


peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

sim_filter_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params, output_folder="sims/gas_acc/test/")
sim_filter_alpha_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2, output_folder="sims/gas_acc/test/")
sim_filter_alpha_flux_alphafrag = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3, output_folder="sims/gas_acc/test/")


In [ ]:
sim_filter_flux_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params)
sim_filter_alpha_flux_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2)
sim_filter_alpha_flux_alphafrag_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3)


In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,6))

ls = ['--','-']
#title = ["$F_{\mathrm{ice}} = 50 \% F_0 $", "$F_{\mathrm{ice}} = 50 \% F_0 $ + "+r"$\alpha_z = 10^{-3}$ ", "$F_{\mathrm{ice}} = 50 \% F_0 $ + "+r"$\alpha_z, \alpha_{\mathrm{frag}} = 10^{-3}$"]
title = ["Irradiated disc", "Surface-heated disc", "Midplane-heated disc"]

cmap = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))
# # nofilter for 3 plots
# for i, (sim, params) in enumerate(zip([ sim_filter_alpha_flux_nofil, sim_filter_alpha_flux_alphafrag_nofil],[ params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
#     plot.plot_growth_track_timescale(fig, axs[i+1], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=False,
#                                       add_cbar=False, nofilter=True)

# # nofilter for just the third plot
# for i, (sim, params) in enumerate(zip([ sim_filter_alpha_flux_alphafrag_nofil],[params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
#     plot.plot_growth_track_timescale(fig, axs[2], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=False,
#                                       add_cbar=True, nofilter=True)

# filter for the 3 plots
for i, (sim, params) in enumerate(zip([sim_filter_flux, sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag],[params_flux, params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
    plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
                                      add_cbar=True if i ==0 else False, nofilter=False)
    axs[i].set_title(title[i], fontsize = 25)
    pos = np.geomspace(4e-3, 1e2, num = 1000)
    M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
    M_iso_in=M_peb_iso(pos, sim_params.t_in, params)
    axs[i].fill_between(pos, M_iso_fin, M_iso_in,  color='slateblue', alpha=0.1, zorder=10)
    # axs[i].axvline(iceline(sim_params.t_fin, 170, params), color= 'lightblue', linestyle = '-.')
    # axs[i].axvline(iceline(sim_params.t_in, 170, params), color= 'lightblue', linestyle = '-.')
    # axs[i].axvspan(iceline(sim_params.t_in, 170, params), iceline(sim_params.t_fin, 170, params),  color='lightblue', alpha=0.1)

for i in range(3):
    axs[i].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    plot.HD219134(axs[i], color = 'red')
    plot.solar_system(axs[i], color = 'green')

fig.subplots_adjust(wspace=0.25)  # Increase the value to add more space

plt.savefig("figures/5_planets_filter_nofilter_5Myr_newMdot", bbox_inches = 'tight')

## ICELINE PLOT 

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'iceline_radius': None,
                'v_frag':(1*u.m/u.s).to(u.au/u.Myr).value,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                }

t_in = 0.2
t_fin = 5
# a_p0_in = np.geomspace(3, 0.1, num=5)
# # Combine the single value 20 with the generated array
# a_p0 = np.concatenate(([20], a_p0_in))
a_p0 = np.geomspace(60, 0.1, num = 5)
print(a_p0)
t0= ([t_in] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work

sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

params_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=False, iceline_alpha_frag_change=False)
#params_alpha_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=False)
params_alpha_frag = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=False, iceline_alpha_frag_change=True)
params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=True)

# params_flux = code_gas.Params(**params_dict, H_r_model='irradiated')
# params_alpha_flux = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5)
# params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1)

m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_flux), params_flux)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)

# m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux), params_alpha_flux)
# sim_params2 = code_gas.SimulationParams(**sim_params_dict, m0 = m02)

m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_frag), params_alpha_frag)
sim_params2 = code_gas.SimulationParams(**sim_params_dict, m0 = m02)


m03 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux_alphafrag), params_alpha_flux_alphafrag)
sim_params3 = code_gas.SimulationParams(**sim_params_dict, m0 = m03)


peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

sim_filter_flux = code_gas.simulate_euler(migration=False, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params)
#sim_filter_alpha_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2)
sim_filter_alpha_frag = code_gas.simulate_euler(migration=False, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_frag, sim_params=sim_params2)
sim_filter_alpha_flux_alphafrag = code_gas.simulate_euler(migration=False, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3)


In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,6))

ls = ['--','-']
#title = ["$F_{\mathrm{ice}} = 50 \% F_0 $", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_z = 10^{-3}$ ", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_z, \alpha_{\mathrm{frag}} = 10^{-3}$"]
title = ["$F_{\mathrm{ice}} = 50 \% F_0 $", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_{\mathrm{frag}} = 10^{-3}$ ", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_z, \alpha_{\mathrm{frag}} = 10^{-3}$"]

cmap = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))

# filter for the 3 plots
for i, (sim, params) in enumerate(zip([sim_filter_flux, sim_filter_alpha_frag, sim_filter_alpha_flux_alphafrag],[params_flux, params_alpha_frag, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):

#for i, (sim, params) in enumerate(zip([sim_filter_flux, sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag],[params_flux, params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
    plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=False, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
                                      add_cbar=True if i ==0 else False, nofilter=False)
    axs[i].set_title(title[i], fontsize = 25)
    pos = np.geomspace(4e-3, 1e2, num = 1000)
    M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
    M_iso_in=M_peb_iso(pos, sim_params.t_in, params)
    axs[i].fill_between(pos, M_iso_fin, M_iso_in,  color='slateblue', alpha=0.1, zorder=10)
    axs[i].axvline(iceline(sim_params.t_fin, 170, params), color= 'lightblue', linestyle = '-.')
    axs[i].axvline(iceline(sim_params.t_in, 170, params), color= 'lightblue', linestyle = '-.')
    axs[i].axvspan(iceline(sim_params.t_in, 170, params), iceline(sim_params.t_fin, 170, params),  color='lightblue', alpha=0.1)

for i in range(3):
    axs[i].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    plot.HD219134(axs[i], color = 'red')
    plot.solar_system(axs[i], color = 'green')

fig.subplots_adjust(wspace=0.25)  # Increase the value to add more space

plt.savefig("figures/5_planets_iceline_5Myr_alphafrag_nomig", bbox_inches = 'tight')

In [ ]:
const.m_p/const.k_B

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'M_dot_star': (1e-8*u.M_sun/u.yr).to(u.M_earth/u.Myr).value,
               'iceline_radius': None,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'iceline_alpha_change': False,
                'iceline_flux_change': False
                }

r = np.geomspace(1e-2, 1e2, num = 1000)
color = ['mediumaquamarine', 'gold', 'orange', 'slateblue', 'purple']
t = 0.1
ls = ['-', '-.']
ls_gas = ['--', ':']

fig, axs = plt.subplots(1,3, figsize = (18,6))
for j, model in enumerate(["irradiated", "Ida_mixed", "Liu_mixed"]):
    for k, alpha in enumerate([1e-2, 1e-3]):
        params = code_gas.Params(**params_dict, H_r_model = model, alpha = alpha)
        axs[0].loglog(r, H_R(r, t, params), color = color[j], label = model+r"$, \alpha$="+str(params.alpha) if k == 0 else None, linestyle = ls[k])
        axs[1].loglog(r, T_mid(r, t, params), color = color[j], label = model+r"$, \alpha$="+str(params.alpha) if j == 0 else None, linestyle = ls[k])
        print(T_mid(1, t, params), str(params.H_r_model))
        print("stokes", st_frag_drift(r, t, params))
        axs[2].loglog(r, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)*(u.M_earth/u.au**2).to(u.g/u.cm**2), color = color[j], label = "$\Sigma_{\mathrm{peb}}$"+r"$, \alpha$="+str(params.alpha) if j ==0 else None, linestyle = ls[k])
        axs[2].loglog(r, sigma_gas_steady_state(r, t, params)*(u.M_earth/u.au**2).to(u.g/u.cm**2), color = color[j], label = "$\Sigma_{\mathrm{gas}}$"+r"$, \alpha$="+str(params.alpha) if j ==0 else None, linestyle = ls_gas[k])

epsilon_el = [1e-2, 1]
epsilon_heat = [0.5, 1]
for j, (e_el, e_h) in enumerate(zip(epsilon_el, epsilon_heat)):
    for k, alpha in enumerate([1e-2, 1e-3]):

        params = code_gas.Params(**params_dict, H_r_model = "Lambrechts_mixed", epsilon_el=e_el ,epsilon_heat=e_h, alpha = alpha)
        axs[0].loglog(r, H_R(r, t, params), color = color[j+3], label = params.H_r_model + ", $\epsilon_{el}$ = "+str(e_el)+", $\epsilon_{heat}$ = "+str(e_h)+r"$, \alpha$="+str(params.alpha) if k == 0 else None, linestyle = ls[k]) 
        print("epsilon_el=", params.epsilon_el, "epsilon_heat=",  params.epsilon_heat)
        axs[1].loglog(r, T_mid(r, t, params), color = color[j+3], linestyle = ls[k])
        axs[1].loglog(r, T_MMSN(r, params), color = 'black')
        print(T_mid(1, t, params), str(params.H_r_model))
        axs[2].loglog(r, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)*(u.M_earth/u.au**2).to(u.g/u.cm**2), color = color[j+3], linestyle = ls[k])
        axs[2].loglog(r, sigma_gas_steady_state(r, t, params)*(u.M_earth/u.au**2).to(u.g/u.cm**2), color = color[j+3],linestyle = ls_gas[k])



axs[1].axhline(y = 170, color = 'lightblue', linestyle = ':', alpha = 0.5, zorder = 0, label = "Water ice line")
axs[1].axhline(y = 1200, color = 'red', linestyle = ':', alpha = 0.5, zorder = 0, label = "Si line")
axs[2].axhline(1700, color = 'grey', linestyle = ':', alpha = 0.5, zorder = 0, label = "1700 $\mathrm{g/cm^2}$")

for i in range(3):
    axs[i].set_xlabel('r[AU]', size = 15)
    axs[i].tick_params(axis = "both", length = 8, which = "minor")
    axs[i].tick_params(axis = "both", length = 10, which = "major")
    axs[i].legend()
    axs[i].axvline(1, color = 'lightgrey', linestyle = '--', alpha = 0.5, zorder = 0)
axs[1].set_ylabel("T [K]",  size = 15)
axs[0].set_ylabel('H/R', size = 15)
axs[2].set_ylabel('$\Sigma \mathrm{[g/cm^2]}$', size = 15)

axs[2].set_ylim(1e-1, 1e7)
axs[1].set_ylim(1e1, 1e6)
axs[0].set_ylim(1e-3, 3e-1)

axs[0].set_title(" H/R profiles for t = "+str(t), fontsize = 15)
axs[1].set_title(" Midplane temperatures for t = "+str(t), fontsize = 15)
axs[2].set_title(" Surface densities for t = "+str(t), fontsize = 15)


plt.tight_layout()
plt.savefig('figures/H_R_t0_alphas_1e-8')

In [ ]:
print("stokes 1 m/s", st_frag_drift(1, 0.1, params)**(1/2))
print("stokes 1m/s", 13*st_frag_drift(1, 0.1, params)**(-1/3))
params2 = code_gas.Params(H_r_model='irradiated', v_frag=(10*u.m/u.s).to(u.au/u.Myr).value, alpha=1e-2, alpha_z=1e-4, alpha_frag=1e-4)
print("stokes 10 m/s", st_frag_drift(1, 0.1, params2)**(1/2))
print("stokes 10 m/s", 13*st_frag_drift(1, 0.1, params2)**(-1/3))


In [ ]:
(10**(-2))**(-2)

In [ ]:
from matplotlib import pyplot as plt, ticker as mticker

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                #'M_dot_star': 1e-8*const.M_sun.cgs/(1*u.yr).to(u.s),
                'iceline_radius': None,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'alpha': 1e-2,
                'v_frag': (1 * u.m/u.s).to(u.cm/u.s),
                'iceline_alpha_change': False,
                'iceline_flux_change': False
                }

r = (np.geomspace(3e-3, 1e2, num = 1000)*u.au).to(u.cm)
color = [ 'slateblue', 'purple', 'gold', 'orange', 'mediumaquamarine']
t = 0.1 #will be ignored because I set the Mdot_*
t_in = 0.1
t_fin = 5

model_label = [ "Ida et al. 2016", "Liu et al. 2019", "irradiated"]
ls = '-'
ls_gas = '-.'

fig = plt.figure( figsize = (9, 11))
gs = fig.add_gridspec(2, 1, hspace=0, wspace=0)
(axs0, axs1) = gs.subplots(sharex='col')


epsilon_el = [1e-2, 1]
epsilon_heat = [0.5, 1]
lam_mod=['surface-heated', 'midplane-heated']
for j, (e_el, e_h) in enumerate(zip(epsilon_el, epsilon_heat)):
    params = code_gas.Params(**params_dict, H_r_model = "Lambrechts_mixed", epsilon_el=e_el ,epsilon_heat=e_h)
    T = 1200
    si_subl_line = iceline(t, T, params)
    si_subl_point = np.argmin(r < si_subl_line)

    axs1.scatter(si_subl_line.to(u.au).value, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point], color = 'red', label = "Si sublimation front" if j==0 else None, zorder = 20)

    axs0.loglog(r.to(u.au), H_R(r, t, params), color = color[j], 
                  label = "this study (" + lam_mod[j]+"), \n$\epsilon_{el}$ = "+str(e_el)+", $\epsilon_{heat}$ = "+str(e_h), linestyle = ls) 
    print(params.epsilon_el, params.epsilon_heat)

    axs1.loglog(r.to(u.au)[si_subl_point:], sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point:], color = color[j], linestyle = ls)
    axs1.loglog(r.to(u.au), sigma_gas_steady_state(r, t, params), color = color[j], linestyle = ls_gas)
    axs1.loglog(r.to(u.au), MMSN(r), color = 'black', linestyle = ':', alpha = 0.5, zorder = 0, label = "MMSN, Hayashi et al. 1981" if j == 0 else None)

for j, model in enumerate(["Ida_mixed", "Liu_mixed","irradiated"]):
    params = code_gas.Params(**params_dict, H_r_model = model)
    print(params.H_r_model)
    T = 1200
    si_subl_line = iceline(t, T, params)
    si_subl_point = np.argmin(r < si_subl_line)
    axs1.scatter(si_subl_line.to(u.au).value, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point], color = 'red',  zorder = 20)
    axs0.loglog(r.to(u.au), H_R(r, t, params), color = color[j+2], label = model_label[j], linestyle = ls)
    axs1.loglog(r.to(u.au)[si_subl_point:], sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point:], 
                  color = color[j+2], label = "$\Sigma_{\mathrm{peb}}$" if j == 2 else None, linestyle = ls)
    axs1.loglog(r.to(u.au), sigma_gas_steady_state(r, t, params), color = color[j+2], 
                  label = "$\Sigma_{\mathrm{gas}}$" if j == 2 else None, linestyle = ls_gas)
 


for axs in [axs0, axs1]:
    axs1.set_xlabel('r [AU]', size = 25)
    axs.tick_params(axis = "both", length = 10, which = "minor", direction = 'in')
    axs.tick_params(axis = "both", length = 15, which = "major", direction = 'in', labelsize = 15)
    axs.legend(fontsize = 14)
    #axs[i].axvline(1, color = 'lightgrey', linestyle = ':', alpha = 0.5, zorder = 0)
    axs.axvline(r_magnetic_cavity(t_in, params).to(u.au).value, linestyle = '-.', color = 'grey', alpha = 0.5)
    axs.set_xlim(3e-3, 1e2)
    axs.yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs.yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))

axs0.text(r_magnetic_cavity(t_in, params).to(u.au).value+4e-3, 1.5e-3, '$r_{\mathrm{mag. cav.}}$', fontsize=15, color='grey', rotation=90, rotation_mode='anchor', ha='left', va='bottom', zorder = 0)
axs1.text(r_magnetic_cavity(t_in, params).to(u.au).value+4e-3, 1e-1, '$r_{\mathrm{mag. cav.}}$', fontsize=15, color='grey', rotation=90, rotation_mode='anchor', ha='left', va='bottom', zorder = 0)

axs0.set_ylabel('H/r', size = 25)
axs1.set_ylabel('$\Sigma \: \mathrm{[g/cm^2]}$', size = 25)

#axs[2].set_ylim(1e-1, 1e7)
axs0.set_ylim(1e-3, 3e-1)

axs0.legend(loc= (0.50, 0.06), fontsize=14)
axs0.set_title(" $t_0$ = "+str(t.to(u.Myr)), fontsize = 19)
#axs[1].set_title(" Surface densities at $t_0$ = "+str(t.to(u.Myr)), fontsize = 19)
# axs[0].set_title(" H/R profiles for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)
# axs[1].set_title(" Midplane temperatures for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)
# axs[2].set_title(" Surface densities for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)


plt.tight_layout()
plt.savefig('figures/H_R_sigma_t0_alpha'+str(params.alpha)+".png")